In [1]:
!git clone https://github.com/brendenlake/SCAN.git
%pip install -r requirements.txt --quiet

fatal: destination path 'SCAN' already exists and is not an empty directory.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb
from Encoder import EncoderRNN
from Decoder import DecoderRNN
from Dataloader import Dataloader, get_dataset_path
from Evaluate import get_accuracy, plot_attention, generate_cosine_latex_table, get_cosine_table, get_cosine_table2
import torch
import os
device = "cuda:0" if torch.cuda.is_available() else "cpu"
os.environ['WANDB_NOTEBOOK_NAME'] = 'results.ipynb'
wandb.login()
api = wandb.Api()

wandb: Currently logged in as: hojmax (rl-msps). Use `wandb login --relogin` to force relogin


In [3]:
run_path = "project-group-1/Paper-Implemenation/1niacmmy"
run = api.run(run_path)

In [4]:
files_to_restore = [
    'encoder.pt',
    'decoder.pt',
    'input_lang.pkl',
    'output_lang.pkl',
    'max_length.txt'
]
files = run.files()

for file in files:
    if file.name in files_to_restore:
        file.download(replace=True, root='./saved_models')

config = run.config

In [5]:
dataloader = Dataloader()
dataloader.load('./saved_models')
test_path = get_dataset_path(config['_folder'], config['dataset'], 'test')
test_X, test_Y = dataloader.transform(test_path)

In [6]:
encoder = EncoderRNN(
    RNN_type=config['rnn-type'],
    input_size=config['input_size'],
    hidden_size=config['hidden_units'],
    hidden_layers=config['layers'],
    dropout=config['dropout']
).to(device)
decoder = DecoderRNN(
    RNN_type=config['rnn-type'],
    input_size=config['hidden_units'],
    hidden_size=config['hidden_units'],
    hidden_layers=config['layers'],
    dropout=config['dropout'],
    attention=config['attention'],
    output_size=config['output_size']
).to(device)


In [7]:
encoder.load_state_dict(torch.load('./saved_models/encoder.pt', map_location=device))
decoder.load_state_dict(torch.load('./saved_models/decoder.pt', map_location=device))

<All keys matched successfully>

In [9]:
a,b,c=get_cosine_table2(encoder,dataloader)

Encoding all words...
Word: run
run: 1.0000001192092896
walk: 0.66630619764328
look: 0.6578210592269897
run after run: 0.5222382545471191
walk after run: 0.5188137888908386
look after run: 0.5185701847076416

Word: jump
walk: 0.24458806216716766
run: 0.24216167628765106
look: 0.1954694390296936
turn right: 0.1742076873779297
turn left: 0.1003132238984108
walk after turn right: 0.08469123393297195

Word: run twice
run twice: 1.0
walk twice: 0.8641841411590576
look twice: 0.8578042387962341
run twice and run twice: 0.7139161825180054
run thrice and run twice: 0.7085099220275879
walk thrice and run twice: 0.6941946744918823

Word: jump twice
run twice and run: 0.44663429260253906
look twice and run: 0.4423865079879761
walk twice and run: 0.4376722574234009
run twice and walk: 0.40559035539627075
walk twice and walk: 0.39960363507270813
look twice and walk: 0.39621222019195557



In [10]:
a

['run', 'jump', 'run twice', 'jump twice']

In [11]:
b

[['run', 'walk', 'look', 'run after run', 'walk after run', 'look after run'],
 ['walk', 'run', 'look', 'turn right', 'turn left', 'walk after turn right'],
 ['run twice',
  'walk twice',
  'look twice',
  'run twice and run twice',
  'run thrice and run twice',
  'walk thrice and run twice'],
 ['run twice and run',
  'look twice and run',
  'walk twice and run',
  'run twice and walk',
  'walk twice and walk',
  'look twice and walk']]

In [18]:
import numpy as np

In [72]:
last = []
for j in range(6):
    last.append([])
    for i in range(4):
        # Round float to 2 decimals
        last[j].append(f'{b[i][j]}\quad\hfill {np.round(c[i][j], 2)}')

In [73]:
# Pandas dataframe
import pandas as pd
df = pd.DataFrame(last, columns=['run', 'jump', 'run twice', 'jump twice'])

In [76]:
# df To latex
print(df.to_latex(index=False)
        .replace('\\textbackslash ', '\\')
        .replace('\\\\', '\\\\ \hline')
        .replace('llll', 'l|l|l|l')        
)

\begin{tabular}{l|l|l|l}
\toprule
                           run &                                  jump &                                 run twice &                         jump twice \\ \hline
\midrule
            run\quad\hfill 1.0 &                  walk\quad\hfill 0.24 &                  run twice\quad\hfill 1.0 &  run twice and run\quad\hfill 0.45 \\ \hline
          walk\quad\hfill 0.67 &                   run\quad\hfill 0.24 &                walk twice\quad\hfill 0.86 & look twice and run\quad\hfill 0.44 \\ \hline
          look\quad\hfill 0.66 &                   look\quad\hfill 0.2 &                look twice\quad\hfill 0.86 & walk twice and run\quad\hfill 0.44 \\ \hline
 run after run\quad\hfill 0.52 &            turn right\quad\hfill 0.17 &   run twice and run twice\quad\hfill 0.71 & run twice and walk\quad\hfill 0.41 \\ \hline
walk after run\quad\hfill 0.52 &              turn left\quad\hfill 0.1 &  run thrice and run twice\quad\hfill 0.71 & walk twice and walk\quad\hfill

/var/folders/j8/qdr4bmbd6vv8nydcr40cfdhw0000gn/T/ipykernel_54487/1036477455.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False)


In [12]:
c

[[1.0000001192092896,
  0.66630619764328,
  0.6578210592269897,
  0.5222382545471191,
  0.5188137888908386,
  0.5185701847076416],
 [0.24458806216716766,
  0.24216167628765106,
  0.1954694390296936,
  0.1742076873779297,
  0.1003132238984108,
  0.08469123393297195],
 [1.0,
  0.8641841411590576,
  0.8578042387962341,
  0.7139161825180054,
  0.7085099220275879,
  0.6941946744918823],
 [0.44663429260253906,
  0.4423865079879761,
  0.4376722574234009,
  0.40559035539627075,
  0.39960363507270813,
  0.39621222019195557]]

In [ ]:
a = get_cosine_table(encoder, dataloader)

run vs look: 0.66
run vs walk: 0.67
run vs walk after run: 0.52
run vs run thrice after run: 0.49
run vs run twice after run: 0.49
jump vs run: 0.24
jump vs walk: 0.24
jump vs turn right: 0.17
jump vs look right twice after walk twice: -0.05
jump vs turn right after turn right: -0.02
run twice vs look twice: 0.86
run twice vs run twice and look opposite right thrice: 0.48
run twice vs run twice and run right twice: 0.59
run twice vs run twice and look opposite right twice: 0.62
run twice vs walk twice and run twice: 0.68
jump twice vs walk and walk: 0.36
jump twice vs run and walk: 0.34
jump twice vs walk opposite right and walk: 0.24
jump twice vs look right and walk: 0.20
jump twice vs walk right and walk: 0.22


In [ ]:
a

(                    run                               jump  \
 0                  look                                run   
 1                  walk                               walk   
 2        walk after run                         turn right   
 3  run thrice after run  look right twice after walk twice   
 4   run twice after run        turn right after turn right   
 
                                   run twice                    jump twice  
 0                                look twice                 walk and walk  
 1  run twice and look opposite right thrice                  run and walk  
 2             run twice and run right twice  walk opposite right and walk  
 3   run twice and look opposite right twice           look right and walk  
 4                  walk twice and run twice           walk right and walk  ,
         run      jump  run twice  jump twice
 0  0.657821  0.242162   0.857804    0.364178
 1  0.666306  0.244588   0.482909    0.335655
 2  0.518814  0.174

In [ ]:
import torch

In [ ]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

In [ ]:
cos(
    torch.tensor([1, 2], dtype=torch.float),
    torch.tensor([-2, 1], dtype=torch.float)
)

tensor(0.)

In [ ]:
# data = get_accuracy_across_length(
#     filter=test_Y,  # Across test_Y
#     test_X=test_X,
#     test_Y=test_Y,
#     encoder=encoder,
#     decoder=decoder,
#     input_max_length=dataloader.input_max_length
# )

In [ ]:
accuracy = get_accuracy(
    test_X=test_X,
    test_Y=test_Y,
    encoder=encoder,
    decoder=decoder,
    input_max_length=dataloader.input_max_length
)
print(f'Accuracy: {accuracy}')

KeyboardInterrupt: 

In [ ]:
# set the oracle as True to predict with length oracle
accuracy = get_accuracy(
    test_X=test_X,
    test_Y=test_Y,
    encoder=encoder,
    decoder=decoder,
    input_max_length=dataloader.input_max_length,
    oracle=True
)
print(f'Accuracy: {accuracy}')

# also for across length
# data = get_accuracy_across_length(
#     filter=test_Y,  # Across test_Y
#     test_X=test_X,
#     test_Y=test_Y,
#     encoder=encoder,
#     decoder=decoder,
#     input_max_length=dataloader.input_max_length,
#     oracle=True
# )

In [ ]:
input_string = "look around right thrice and turn left thrice"
plot_attention(
    encoder=encoder,
    decoder=decoder,
    dataloader=dataloader,
    input_string=input_string,
    figsize=(14, 6),
    filename="attention1.png"
)